In [1]:
from src.synthetic_data import generate_synthetic_data

save_path_train = "C:\\Users\\jiayang\\ipynb\\APS_Thesis\\data\\synthetic_data\\synthetic_5class_NoTemp_train.pkl"
save_path_test = "C:\\Users\\jiayang\\ipynb\\APS_Thesis\\data\\synthetic_data\\synthetic_5class_NoTemp_test.pkl"
generate_synthetic_data( k=5, save_path=save_path_train, temperature= 1.0) # for model training
generate_synthetic_data( k=5, save_path=save_path_test, temperature= 1.0 ) # for aps tests

Synthetic Data with 5 classes has been saved at: C:\Users\jiayang\ipynb\APS_Thesis\data\synthetic_data\synthetic_5class_NoTemp_train.pkl
Synthetic Data with 5 classes has been saved at: C:\Users\jiayang\ipynb\APS_Thesis\data\synthetic_data\synthetic_5class_NoTemp_test.pkl


In [2]:
from src.synthetic_data import load_synthetic_data
from torch.utils.data import DataLoader, TensorDataset
import torch

x, true_labels, real_probs = load_synthetic_data(save_path_train)
print(len(x), len(true_labels), len(real_probs)) 

# 8000 for training, 2000 for validation
x_train, true_labels_train = x[:8000], true_labels[:8000]
x_val, true_labels_val = x[8000:], true_labels[8000:]

train_loader = DataLoader(TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(true_labels_train).long()), batch_size=128, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.from_numpy(x_val).float(), torch.from_numpy(true_labels_val).long()), batch_size=256)

10000 10000 10000


In [3]:
from src.synthetic_data import SimplePredictor, train_simple_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimplePredictor(feature_dim=64, n_classes=5)

train_simple_model(model, train_loader, val_loader, epochs=5, lr=0.01, device=device)

Epoch 1/5 | Train Loss: 1.0033, Acc: 0.5919 | Val Loss: 0.5823, Acc: 0.7950
Epoch 2/5 | Train Loss: 0.5558, Acc: 0.8024 | Val Loss: 0.4282, Acc: 0.8460
Epoch 3/5 | Train Loss: 0.4482, Acc: 0.8395 | Val Loss: 0.4675, Acc: 0.8250
Epoch 4/5 | Train Loss: 0.3818, Acc: 0.8646 | Val Loss: 0.4214, Acc: 0.8410
Epoch 5/5 | Train Loss: 0.3376, Acc: 0.8775 | Val Loss: 0.4573, Acc: 0.8485


In [4]:
train_simple_model(model, train_loader, val_loader, epochs=5, lr=0.001, device=device)

Epoch 1/5 | Train Loss: 0.2302, Acc: 0.9165 | Val Loss: 0.3962, Acc: 0.8750
Epoch 2/5 | Train Loss: 0.1505, Acc: 0.9481 | Val Loss: 0.4652, Acc: 0.8790
Epoch 3/5 | Train Loss: 0.1265, Acc: 0.9553 | Val Loss: 0.4993, Acc: 0.8775
Epoch 4/5 | Train Loss: 0.1058, Acc: 0.9620 | Val Loss: 0.5557, Acc: 0.8785
Epoch 5/5 | Train Loss: 0.0958, Acc: 0.9651 | Val Loss: 0.5965, Acc: 0.8730


In [5]:
save_path_model = "C:\\Users\\jiayang\\ipynb\\APS_Thesis\\data\\model_weights\\synthetic_5class_NoTemp_model.pth"
torch.save(model.state_dict(), save_path_model)

In [1]:
import torch
from src.synthetic_data import SimplePredictor
# load model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimplePredictor(feature_dim=64, n_classes=5)
save_path_model = "C:\\Users\\jiayang\\ipynb\\APS_Thesis\\data\\model_weights\\synthetic_5class_NoTemp_model.pth"
model.load_state_dict(torch.load(save_path_model, weights_only=True, map_location=device))
model.to(device)
model.eval()

SimplePredictor(
  (fc1): Linear(in_features=64, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=16, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (out): Linear(in_features=16, out_features=5, bias=True)
)

In [2]:
from src.synthetic_data import SyntheticDataset_and_Probs, check_tvd

save_path_train = "C:\\Users\\jiayang\\ipynb\\APS_Thesis\\data\\synthetic_data\\synthetic_5class_NoTemp_train.pkl"
x, true_labels, real_probs = load_synthetic_data(save_path_train)
dataset = SyntheticDataset_and_Probs(x, true_labels, real_probs)
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)
check_tvd(model, dataloader, device=device)

Total Variation Distance:  1216.5394
Average Total Variation Distance:  0.1217
